<font size=50 color=darkblue>Max Flow LP</font>

# Problem modelling in LP format

## Import necessary modules
- Import function `read_model` from DOcplex to read the Max-Flow LP from a temporary file
- Module `tempfile` is imported to create the temporary file
- Import function `remove` to delete the temporary file after reading it (required for python version < 3.12)

In [ ]:
from docplex.mp.model_reader import read_model
import tempfile
from os import remove

## Max-Flow LP <font size=3>(variables are colored blue)</font>
**Maximize**
### $${\color{blue}y}_{({\color{red}t},{\color{green}s})}$$
**Subject to**
### \begin{align*}
\sum_{(u,v)\in{\color{red}\delta}^{{\color{red}-}}_{(v)}}{\color{blue}y_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{{\color{green}+}}_{(v)}}{\color{blue}y_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\\
0\le {\color{blue}y_{(u,v)}}\le\mu_{(u,v)},\qquad&\forall (u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/mxf.png' width=700/>

## Write the Max-Flow LP model (in LP format), which is assigned to the variable `mxf_str`

In [ ]:
mxf_str = '''
Maximize
 obj: y(7,0)
 
Subject To
 flow_conservation(0): y(7,0) - y(0,1) - y(0,2) - y(0,3) = 0
 flow_conservation(1): y(0,1) - y(1,2) - y(1,4) - y(1,5) = 0
 flow_conservation(2): y(0,2) + y(1,2) + y(6,2) - y(2,3) - y(2,5) = 0
 flow_conservation(3): y(0,3) + y(2,3) - y(3,6) = 0
 flow_conservation(4): y(1,4) - y(4,5) - y(4,7) = 0
 flow_conservation(5): y(1,5) + y(2,5) + y(4,5) - y(5,6) - y(5,7) = 0
 flow_conservation(6): y(3,6) + y(5,6) - y(6,2) - y(6,7) = 0
 flow_conservation(7): y(4,7) + y(5,7) + y(6,7) - y(7,0) = 0

Bounds
 0 <= y(0,1) <= 10
 0 <= y(0,2) <= 5
 0 <= y(0,3) <= 15
 0 <= y(1,2) <= 4
 0 <= y(1,4) <= 9
 0 <= y(1,5) <= 15
 0 <= y(2,3) <= 4
 0 <= y(2,5) <= 8
 0 <= y(6,2) <= 6
 0 <= y(3,6) <= 30
 0 <= y(4,5) <= 15
 0 <= y(4,7) <= 10
 0 <= y(5,6) <= 15
 0 <= y(5,7) <= 10
 0 <= y(6,7) <= 10
End
'''

## To import the LP model to DOcplex
- Store the model to a temporary file
- Have DOcplex read the model from the temporary file

In [ ]:
# Store string to tmp file
with tempfile.TemporaryFile(delete=False) as tmp:
    tmp.write(mxf_str.encode('utf-8'))
    tmp.close()
    # Have DOcplex read the string
    mxf_LP = read_model(filename=tmp.name, model_name='Max Flow')
    # Delete the temporary file
    remove(tmp.name)

## Summarize the model

In [ ]:
mxf_LP.print_information()

## Solve the LP and display the result

In [ ]:
mxf_sol = mxf_LP.solve()
if mxf_sol:
    mxf_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`, `re`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import re

## Extract the link list `E`, link capacities $\mu$, source <code><font color='green'>s</font></code>, and sink <code><font color='red'>t</font></code> from the LP model

In [ ]:
E, mu = [], {}

for var in mxf_LP.iter_variables():
    u, v = re.findall('y\((.+),(.+)\)', var.name)[0]
    E.append(e:=(int(u), int(v)))
    mu[e] = int(var.ub)

t, s = re.findall('y\((.+),(.+)\)', mxf_LP.objective_expr.to_string())[0]
s, t = int(s), int(t)

## Since the link (<font color='red'>t</font>, <font color='green'>s</font>) is imaginary, remove it from the link list `E`

In [ ]:
E.remove((t,s))

## For convenience, extract the solution to a dictionary named `sol_y`

In [ ]:
sol_y = {(u,v): mxf_sol.get_value(f'y({u},{v})') for u,v in E}
sol_y

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

lnk_flow_max = max(sol_y.values())
g.es['flow'] = [sol_y[e] for e in E]
g.es['cap'] = [mu[e] for e in E]
g.es['label'] = ' '
g.es['arrow_size'] = [10 if sol_y[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_y[e] > 0 else None for e in E]
g.es['width'] = [.4*(sol_y[e] + 1) for e in E]
g.es['color'] = ['darkblue' if sol_y[e] > 0 else None for e in E]
g.es['flow_label_color'] = ['blue' if sol_y[e] > 0 else 'black' for e in E]
g.es['cap_label_color']  = ['red' if sol_y[e] > 0 else 'black' for e in E]
g.es['label_size'] = [10 if sol_y[e] > 0 else 7 for e in E]

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('fr'), edge_label=g.es['label'], target=ax)

for e, edge_label in zip(g.es, p.get_edge_labels()):
    txt = ax.annotate(f'{e["flow"]:g}', xycoords=edge_label, xy=(-5,0), color=e['flow_label_color'], backgroundcolor='white', size=e['label_size'])
    txt =  ax.annotate(f' | ', xycoords=txt, xy=(1,0), color='k', size=e['label_size'], verticalalignment='bottom')
    txt =  ax.annotate(f'{e["cap"]}', xycoords=txt, xy=(1,0), color=e['cap_label_color'], backgroundcolor='white', size=e['label_size'], verticalalignment='bottom')
    
plt.show()